In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import random

In [ ]:
#Action value function Q_t(a): Incremental approach
##Q_t+1(a) = Q_t(a) + (1/N_t(a))*(R_t(a) - Q_t(a))
def sample(X,t,A,a,Q,step):
    ind = (A[t] == a)
    if ind == 1:
        Q = Q + (1/step[a])*(X[t,a]-Q)

    return Q

In [ ]:
#Sampling from the softmax distribution function - for Gradient Bandit
def softmax(H):   #H(a) = preference
    n = len(H)
    p = np.zeros((n))
    for a in range(n):
        p[a] = np.exp(H[a])/np.sum(np.exp(H))
    p = list(p)
    select = np.asscalar(np.random.choice(a=n,size=1,p=p))

    return select, p #returns action, prob

In [ ]:
#1. Greedy method
def Greedy():
    s = n*2 #randomly start first s actions
    A = np.zeros((t)) 
    step = np.zeros((n))
    
    reward_avg1 = np.zeros((m,t))
    reward_avg2 = np.zeros((t))
    
    for k in range(m):
        reward_tot = 0
        
        #initial random selections
        for i in range(s): 
            A[i] = random.randint(0,n-1)
            
        for a in range(n):
            step[a] =  sum(A[:s] == a)
            
        #iterate through the rest of steps
        Q = X[k,s,:] 
        for i in range(s,t):
            for a in range(n):
                Q[a] =  sample(X[k,:,:],i-1,A,a,Q[a],step)
            A[i] = np.argmax(Q) #greedy
            
            A = A.astype(int)
            step[A[i]] += 1
            reward_tot += X[k,i,A[i]]
            reward_avg1[k,i] =  reward_tot/(i+1)
     
    #Compute average reward over all bandits
    for i in range(t):
        reward_avg2[i] = np.mean(reward_avg1[:,i])
    
    return reward_avg2

In [ ]:
#2. Eps-Greedy (fixed eps)
def EpsGreedy(eps):
    s = n*2 #randomly start first s actions
    A = np.zeros((t)) 
    step = np.zeros((n))
    
    reward_avg1 = np.zeros((m,t))
    reward_avg2 = np.zeros((t))
    
    for k in range(m):
        reward_tot = 0
        
        #initial random selections
        for i in range(s): 
            A[i] = random.randint(0,n-1)
            
        for a in range(n):
            step[a] =  sum(A[:s] == a)
            
        #iterate through the rest of steps
        Q = X[k,s,:] 
        for i in range(s,t):
            for a in range(n):
                Q[a] =  sample(X[k,:,:],i-1,A,a,Q[a],step)
            
            #with prob eps, randomly explore
            u = np.asscalar(np.random.uniform(0,1,1))
            if u < eps:
                A[i] = random.randint(0,n-1)
            else:
                A[i] = np.argmax(Q) #eps-greedy
            
            A = A.astype(int)
            step[A[i]] += 1
            reward_tot += X[k,i,A[i]]
            reward_avg1[k,i] =  reward_tot/(i+1)
     
    #Compute average reward over all bandits
    for i in range(t):
        reward_avg2[i] = np.mean(reward_avg1[:,i])

    return reward_avg2

In [ ]:
#3. Eps-greedy (adaptive eps)
def EpsGreedy2(EPS):
    s = n*2 #randomly start first s actions
    A = np.zeros((t)) 
    step = np.zeros((n))
    
    reward_avg1 = np.zeros((m,t))
    reward_avg2 = np.zeros((t))
    
    for k in range(m):
        reward_tot = 0
        
        #initial random selections
        for i in range(s): 
            A[i] = random.randint(0,n-1)
            
        for a in range(n):
            step[a] =  sum(A[:s] == a)
            
        #iterate through the rest of steps
        Q = X[k,s,:] 
        for i in range(s,t):
            for a in range(n):
                Q[a] =  sample(X[k,:,:],i-1,A,a,Q[a],step)
            
            #with prob eps, randomly explore
            u = np.asscalar(np.random.uniform(0,1,1))
            if u < EPS[i]:
                A[i] = random.randint(0,n-1)
            else:
                A[i] = np.argmax(Q) #eps-greedy
            
            A = A.astype(int)
            step[A[i]] += 1
            reward_tot += X[k,i,A[i]]
            reward_avg1[k,i] =  reward_tot/(i+1)
     
    #Compute average reward over all bandits
    for i in range(t):
        reward_avg2[i] = np.mean(reward_avg1[:,i])

    return reward_avg2

In [ ]:
#4. Upper-Confidence-Bound (UCB)
def UCB(c):
    s = n*2 #randomly start first s actions
    A = np.zeros((t)) 
    step = np.zeros((n))
    
    reward_avg1 = np.zeros((m,t))
    reward_avg2 = np.zeros((t))
    
    for k in range(m):
        reward_tot = 0
        
        #initial random selections
        for i in range(s): 
            A[i] = random.randint(0,n-1)
            
        for a in range(n):
            step[a] =  sum(A[:s] == a)
            
        #iterate through the rest of steps
        Q = X[k,s,:] 
        UCB = np.zeros((n))
        for i in range(s,t):
            for a in range(n):
                Q[a] =  sample(X[k,:,:],i-1,A,a,Q[a],step)
                if step[a] == 0:
                    UCB[a] = Q[a]
                else:
                    UCB[a] = Q[a] + c*np.sqrt(np.log(i)/step[a])
            A[i] = np.argmax(UCB)
            
            A = A.astype(int)
            step[A[i]] += 1
            reward_tot += X[k,i,A[i]]
            reward_avg1[k,i] =  reward_tot/(i+1)
     
    #Compute average reward over all bandits
    for i in range(t):
        reward_avg2[i] = np.mean(reward_avg1[:,i])

    return reward_avg2         

In [ ]:
#5. Gradient Bandit
def GB(alp):
    step = np.zeros((n))    
    reward_avg1 = np.zeros((m,t))
    reward_avg2 = np.zeros((t))
    
    for k in range(m):
        reward_tot = 0
        A = np.zeros((t)) 
        
        #First step of GB
        H = X[k,0,:]
        A0 = softmax(H)[0]
        A[0] = A0
        A = A.astype(int)
        pi = softmax(H)[1]        
        for a in range(n):
            if a == A0:
                H[a] = H[a] + alp*(1-pi[a])*X[k,0,a]
            else:
                H[a] = H[a] - alp*pi[a]*X[k,0,a]       
        for a in range(n):
            step[a] = (A[0] == a)        
        reward_tot += X[k,0,A[0]]
        reward_avg1[k,0] =  reward_tot
        
        #After first step of GB
        Q = X[k,1,:]
        for i in range(1,t):
            pi = softmax(H)[1]
            Ai = softmax(H)[0]
            A[i] = Ai
            A = A.astype(int)
            reward_tot += X[k,i,A[i]]
            reward_avg1[k,i] =  reward_tot/(i+1)
            step[A[i]] += 1
            
            for a in range(n):
                Q[a] = sample(X[k,:,:],i-1,A,a,Q[a],step)
                if a == A[i]:
                    H[a] = H[a] + alp*(1-pi[a])*(X[k,i,a]-Q[a])
                else:
                    H[a] = H[a] - alp*pi[a]*(X[k,i,a]-Q[a]) 
             
    #Compute average reward over all bandits
    for i in range(t):
        reward_avg2[i] = np.mean(reward_avg1[:,i])

    return reward_avg2        

In [ ]:
#6. Gradient Bandit - Adaptive Step Size
def GB2(ALP):
    step = np.zeros((n))    
    reward_avg1 = np.zeros((m,t))
    reward_avg2 = np.zeros((t))
    
    for k in range(m):
        reward_tot = 0
        A = np.zeros((t)) 
        
        #First step of GB
        H = X[k,0,:]
        A0 = softmax(H)[0]
        A[0] = A0
        A = A.astype(int)
        pi = softmax(H)[1]        
        for a in range(n):
            if a == A0:
                H[a] = H[a] + ALP[0]*(1-pi[a])*X[k,0,a]
            else:
                H[a] = H[a] - ALP[0]*pi[a]*X[k,0,a]       
        for a in range(n):
            step[a] = (A[0] == a)        
        reward_tot += X[k,0,A[0]]
        reward_avg1[k,0] =  reward_tot
        
        #After first step of GB
        Q = X[k,1,:]
        for i in range(1,t):
            pi = softmax(H)[1]
            Ai = softmax(H)[0]
            A[i] = Ai
            A = A.astype(int)
            reward_tot += X[k,i,A[i]]
            reward_avg1[k,i] =  reward_tot/(i+1)
            step[A[i]] += 1
            
            for a in range(n):
                Q[a] = sample(X[k,:,:],i-1,A,a,Q[a],step)
                if a == A[i]:
                    H[a] = H[a] + ALP[i]*(1-pi[a])*(X[k,i,a]-Q[a])
                else:
                    H[a] = H[a] - ALP[i]*pi[a]*(X[k,i,a]-Q[a]) 
             
    #Compute average reward over all bandits
    for i in range(t):
        reward_avg2[i] = np.mean(reward_avg1[:,i])

    return reward_avg2

In [ ]:
#Set parameters
m,n,t,eps1,eps2,c,alp = 5,10,1000,0.1,0.01,1.96,0.001
EPS = np.sort(np.arange(5,5+t,1)/10000)[::-1]
ALP = np.sort(np.arange(1,5+t,1)/10000)[::-1]
step = np.arange(0,t,1)[n*2:]

In [ ]:
#Scenario 1.  Noisy continous reward
X = np.empty((m,t,n)) #Reward matrix
mu1,sigma1 = 0,1
mu2,sigma2 = 0,2

for k in range(m): #averaged over m bandits
    qa = np.random.normal(mu1,sigma1,n) #True action values
    for j in range(t): #1000 time steps
        X[k,j,:] = qa + np.random.normal(mu2,sigma2,n) #Noisy reward

In [ ]:
result1 = Greedy()
result2 = EpsGreedy(eps1)
result3 = EpsGreedy(eps2)
result4 = EpsGreedy2(EPS)
result5 = UCB(c)
result6 = GB(alp)
result7 = GB2(ALP)

In [ ]:
#Average reward plot
fig = plt.figure()
plt.plot(step,result1[n*2:],'r--',label='Greedy')
plt.plot(step,result2[n*2:],'y--',label='Eps = 0.1')
plt.plot(step,result3[n*2:],'g--',label='Eps = 0.01')
plt.plot(step,result4[n*2:],'b--',label='Adaptive Eps')
plt.plot(step,result5[n*2:],'m--',label='UCB')
plt.plot(step,result6[n*2:],'c--',label='Gradient (fixed)')
plt.plot(step,result7[n*2:],'k--',label='Gradient (adaptive)')
plt.title('Average Reward Plot', size=16, fontweight='bold')
plt.xlabel('Time Steps', fontsize=18)
plt.ylabel('Average Reward', fontsize=18)
plt.legend(loc='lower right')

In [ ]:
#Set parameters
m,n,t,eps1,eps2,c,alp = 5,4,1000,0.1,0.01,1,0.05
EPS = np.sort(np.arange(5,5+t,1)/10000)[::-1]
ALP = np.sort(np.arange(5,5+t,1)/10000)[::-1]
step = np.arange(0,t,1)[n*2:]

#Scenario 2.  Bernoulli Bandit: each reward can be either success or failure
X = np.zeros((m,t,n)) #Reward matrix

for k in range(m): #averaged over m bandits
    p = [0.5,0.7,0.4,0.3]
    for j in range(t): #1000 time steps
        X[k,j,:] = np.random.binomial(n=1, p = p, size = (1,n)) #Noisy reward

In [ ]:
result1 = Greedy()
result2 = EpsGreedy(eps1)
result3 = EpsGreedy(eps2)
result4 = EpsGreedy2(EPS)
result5 = UCB(c)
result6 = GB(alp)
result7 = GB2(ALP)

In [ ]:
#Average reward plot
fig = plt.figure()
plt.plot(step,result1[n*2:],'r--',label='Greedy')
plt.plot(step,result2[n*2:],'y--',label='Eps = 0.1')
plt.plot(step,result3[n*2:],'g--',label='Eps = 0.01')
plt.plot(step,result4[n*2:],'b--',label='Adaptive Eps')
plt.plot(step,result5[n*2:],'m--',label='UCB')
plt.plot(step,result6[n*2:],'c--',label='Gradient (fixed)')
plt.plot(step,result7[n*2:],'k--',label='Gradient (adaptive)')
plt.title('Average Reward Plot', size=16, fontweight='bold')
plt.xlabel('Time Steps', fontsize=18)
plt.ylabel('Average Reward', fontsize=18)
plt.legend(loc='lower right')